In [4]:
import torch
import torch.nn as nn
from torch.nn import functional as F

In [5]:
# hyperparameters
batch_size = 16 # how many independent sequences will we process in parallel?
block_size = 32 # what is the maximum context length for predictions?
max_iters = 50000
eval_interval = 100
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 64
n_head = 4
n_layer = 2
ff_dropout = 0.2
attn_dropout = 0.2

In [6]:
torch.manual_seed(1337)

# wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
with open('./input/tinyshakespeare.txt', 'r', encoding='utf-8') as f:
    text = f.read()

# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

In [7]:
# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(attn_dropout)

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)   # (B,T,C)
        q = self.query(x) # (B,T,C)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * C**-0.5 # (B, T, C) @ (B, C, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,C)
        out = wei @ v # (B, T, T) @ (B, T, C) -> (B, T, C)
        return out

class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(attn_dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(ff_dropout),
        )

    def forward(self, x):
        return self.net(x)

class FlexibleBlock(nn.Module):
    """A Transformer block that can adapt between single and double layer configurations."""
    
    def __init__(self, n_embd, n_head, res1_dropout, res2_dropout, res4_dropout=None, use_double_layers=False):
        super().__init__()
        head_size = n_embd // n_head
        self.use_double_layers = use_double_layers
        # Store dropout rates as instance attributes
        self.res1_dropout = res1_dropout
        self.res2_dropout = res2_dropout
        self.res4_dropout = res4_dropout if res4_dropout is not None else 0  # Ensure there's a default value
        
        self.sa1 = MultiHeadAttention(n_head, head_size)
        self.ffwd1 = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)
        self.dropout1 = nn.Dropout(res1_dropout)
        self.dropout2 = nn.Dropout(res2_dropout)
        
        if self.use_double_layers:
            self.sa2 = MultiHeadAttention(n_head, head_size)
            self.ffwd2 = FeedFoward(n_embd)
            self.ln3 = nn.LayerNorm(n_embd)
            self.ln4 = nn.LayerNorm(n_embd)
            self.res4_dropout = res4_dropout
            self.dropout4 = nn.Dropout(res4_dropout) if res4_dropout is not None else None
        
    def forward(self, x):
        x = self.ln1(x)
        y = self.dropout1(x) * (1 - self.res1_dropout) + self.sa1(x)
        
        y = self.ln2(y)
        z = self.dropout2(x) * (1 - self.res2_dropout) + self.dropout1(y) * (1 - self.res1_dropout) + self.ffwd1(y)
        
        if self.use_double_layers:
            z = self.ln3(z)
            a = self.dropout1(z) * (1 - self.res1_dropout) + self.sa2(z)
            a = self.ln4(a)
            z = self.dropout4(x) * (1 - self.res4_dropout) + self.dropout2(z) * (1 - self.res2_dropout) + self.dropout1(a) * (1 - self.res1_dropout) + self.ffwd2(a)
        
        return z


# super simple bigram model
class BigramLanguageModel(nn.Module):

    def __init__(self, res1_dropout=0.0, res2_dropout=0.0, res4_dropout=0.0, use_double_layers=False):
        super().__init__()
        self.vocab_size = vocab_size
        self.n_embd = n_embd
        self.block_size = block_size
        self.device = device
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        
        # 使用 FlexibleBlock 而不是 Block
        self.blocks = nn.Sequential(*[FlexibleBlock(n_embd, n_head, res1_dropout, res2_dropout, res4_dropout, use_double_layers) for _ in range(n_layer)])
        
        self.ln_f = nn.LayerNorm(n_embd)
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx)  # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=self.device))  # (T,C)
        x = tok_emb + pos_emb  # (B,T,C)
        x = self.blocks(x)  # (B,T,C)
        x = self.ln_f(x)  # (B,T,C)
        logits = self.lm_head(x)  # (B,T,vocab_size)

        if targets is not None:
            logits = logits.view(-1, self.vocab_size)  # (B*T, vocab_size)
            targets = targets.view(-1)  # (B*T,)
            loss = F.cross_entropy(logits, targets)
        else:
            loss = None

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # Assuming idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            idx_cond = idx[:, -self.block_size:]  # Crop idx to the last block_size tokens
            logits, _ = self(idx_cond)  # Get the predictions
            logits = logits[:, -1, :]  # Focus only on the last time step
            probs = F.softmax(logits, dim=-1)  # Apply softmax to get probabilities
            idx_next = torch.multinomial(probs, num_samples=1)  # Sample from the distribution
            idx = torch.cat((idx, idx_next), dim=1)  # Append sampled index to the running sequence
        return idx






In [8]:
import json
import os
import time

# Example: Different dropout configurations
dropout_configurations = [
    {"res1_dropout": 0.025, "res2_dropout": 0.99, "res4_dropout": 0.0, "use_double_layers": False},
    {"res1_dropout": 0.1, "res2_dropout": 0.99, "res4_dropout": 0.0, "use_double_layers": False},
    {"res1_dropout": 0.2, "res2_dropout": 0.99, "res4_dropout": 0.0, "use_double_layers": False},
    {"res1_dropout": 0.5, "res2_dropout": 0.99, "res4_dropout": 0.0, "use_double_layers": False},

    {"res1_dropout": 0.025, "res2_dropout": 0.1, "res4_dropout": 0.0, "use_double_layers": False},
    {"res1_dropout": 0.025, "res2_dropout": 0.2, "res4_dropout": 0.0, "use_double_layers": False},
    {"res1_dropout": 0.025, "res2_dropout": 0.5, "res4_dropout": 0.0, "use_double_layers": False},
    {"res1_dropout": 0.025, "res2_dropout": 0.99, "res4_dropout": 0.0, "use_double_layers": False},

    {"res1_dropout": 0.025, "res2_dropout": 0.99, "res4_dropout": 0.1, "use_double_layers": True},
    {"res1_dropout": 0.025, "res2_dropout": 0.99, "res4_dropout": 0.2, "use_double_layers": True},
    {"res1_dropout": 0.025, "res2_dropout": 0.99, "res4_dropout": 0.5, "use_double_layers": True},
    {"res1_dropout": 0.025, "res2_dropout": 0.99, "res4_dropout": 0.99, "use_double_layers": True}
]

for config in dropout_configurations:
    print(f"Training model with configuration: {config}")

    # Start timing
    start_time = time.time()
    
    # Instantiate the model with the current configuration
    model = BigramLanguageModel(
        res1_dropout=config['res1_dropout'],
        res2_dropout=config['res2_dropout'],
        res4_dropout=config['res4_dropout'],
        use_double_layers=config['use_double_layers']
    ).to(device)
    optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
    
    # Store the loss values for the current configuration
    train_losses = []
    val_losses = []

    for iter in range(max_iters):
        # Regularly evaluate the loss on training and validation sets
        if iter % eval_interval == 0 or iter == max_iters - 1:
            losses = estimate_loss()  # Ensure this function uses the model from the current loop
            train_losses.append(losses['train'].item())
            val_losses.append(losses['val'].item())
            print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")
        
        # Sample a batch of data
        xb, yb = get_batch('train')
        
        # Calculate the loss
        logits, loss = model(xb, yb)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # End timing
    end_time = time.time()
    training_time = end_time - start_time  # Calculate total training time

    # Construct the path and filename for saving results
    config_str = "_".join(f"{k}={v}" for k, v in config.items())
    folder_path = "./loss_time/res_all"
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
    loss_file_path = os.path.join(folder_path, f"{config_str}_losses.json")
    time_file_path = os.path.join(folder_path, f"{config_str}_training_time.json")
    
    # Save results
    with open(loss_file_path, 'w') as f:
        json.dump({"train_losses": train_losses, "val_losses": val_losses}, f)
    with open(time_file_path, 'w') as f:
        json.dump({"training_time": training_time}, f)
    
    print(f"Results for configuration {config_str} saved.")



Training model with configuration: {'res1_dropout': 0.025, 'res2_dropout': 0.99, 'res4_dropout': 0.0, 'use_double_layers': False}
step 0: train loss 4.2832, val loss 4.2867
step 100: train loss 2.7362, val loss 2.7493
step 200: train loss 2.5569, val loss 2.5523
step 300: train loss 2.4906, val loss 2.4914
step 400: train loss 2.4569, val loss 2.4551
step 500: train loss 2.4077, val loss 2.4069
step 600: train loss 2.3587, val loss 2.3591
step 700: train loss 2.3319, val loss 2.3434
step 800: train loss 2.2903, val loss 2.2880
step 900: train loss 2.2484, val loss 2.2661
step 999: train loss 2.2237, val loss 2.2435
Results for configuration res1_dropout=0.025_res2_dropout=0.99_res4_dropout=0.0_use_double_layers=False saved.
Training model with configuration: {'res1_dropout': 0.1, 'res2_dropout': 0.99, 'res4_dropout': 0.0, 'use_double_layers': False}
step 0: train loss 4.3015, val loss 4.2993
step 100: train loss 2.7454, val loss 2.7476
step 200: train loss 2.5711, val loss 2.5682
step 

In [10]:
import json
import matplotlib.pyplot as plt
import os

# Specify the folder path where the loss JSON files are stored
folder_path = "./loss_time/res_all"
# Specify the path to save the visualization result
visualization_save_path = "./visualization/res_all_loss_curves.png"

# Use a pattern or a specific part of the file name to identify loss files
loss_files = [f for f in os.listdir(folder_path) if 'losses' in f]

# Visualize the loss curves for each dropout configuration
plt.figure(figsize=(10, 6))

for file in loss_files:
    file_path = os.path.join(folder_path, file)
    
    # Read the loss values from the JSON file
    with open(file_path, 'r') as f:
        losses = json.load(f)
    
    # Extract the dropout values from the file name
    parts = file.replace('res1_dropout=', '').replace('res2_dropout=', '').replace('res4_dropout=', '').replace('use_double_layers=False_losses', '').split('_')
    res1_dropout, res2_dropout, res4_dropout = parts[:3]
    dropout_label = f"res1={res1_dropout}, res2={res2_dropout}, res4={res4_dropout}"
    
    # Plot the validation loss curves
    plt.plot(losses['val_losses'], label=f"{dropout_label}")

plt.title("Loss Curves for Different Dropout Configurations")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()
plt.grid(True)

# Save the visualization
plt.savefig(visualization_save_path)
plt.close()

print(f"Visualization saved to {visualization_save_path}")





Visualization saved to ./visualization/res_all_loss_curves.png
